# ASP工具箱教程2.2：触发器重新排序
### 攥写：<font color="grey">[babycat](https://github.com/babycat262)</font>        技术支持：<font color="grey">[Alian713](https://github.com/Divy1211)</font>

#### 有时候我们在进行场景地图设计时，会给某些功能添加新的触发器，将这些关联触发器位置移到一起。这样就打乱了原先的触发器创建顺序，即后创建的触发器有可能在显示顺序上靠前，先创建的触发有可能靠后。
 - 此处以文档1生成的场景**TEST-触发器删除.aoe2scenario**为例进行说明。该触发器中的触发详情如图所示：

<img src="./images/doc-203.png" width=400 height=200>

场景中 `T1`, `T3`, `T5`, `T7` 触发的顺序被打乱了，并且新建了一个触发器`XS函数定义`，并移动到`T7`的上面

In [1]:
# 设置场景基本信息
import os
from AoE2ScenarioParser.scenarios.aoe2_de_scenario import AoE2DEScenario

# 设置玩家基本信息
infos = {
    "user": "babycat",
    "game_id": 76561198386517457,
}
# 设置场景名称
scx1_name = "TEST-触发器删除"
scx1_out = "TEST-触发器排序"
# 输入，输出路径
src_path = f"./inputs/{scx1_name}.aoe2scenario"
des_path = f"C:/Users/{infos['user']}/Games/Age of Empires 2 DE/{infos['game_id']}/resources/_common/scenario/{scx1_out}.aoe2scenario"

In [2]:
print(src_path)
print(des_path)

./inputs/TEST-触发器删除.aoe2scenario
C:/Users/babycat/Games/Age of Empires 2 DE/76561198386517457/resources/_common/scenario/TEST-触发器排序.aoe2scenario


In [3]:
# ASP读取场景
scenario = AoE2DEScenario.from_file(src_path)
trigger_mgr = scenario.trigger_manager
# 打印触发器数量
print(trigger_mgr.triggers.__len__())


[21:11:20] Reading file: './inputs/TEST-触发器删除.aoe2scenario'
[21:11:20] Reading scenario file finished successfully.

############### Attributes ###############
>>> Game version: 'DE'
>>> Scenario version: 1.53
>>> Scenario variant: 'Age of Empires 2'
##########################################

[21:11:20] Loading scenario structure finished successfully.
[21:11:20] Parsing scenario file...
	✔ FileHeadereHeader data...
	✔ DataHeaderaHeader data...
	✔ Messagesessages data...
	✔ Cinematicsematics data...
	✔ BackgroundImagendImage data...
	✔ PlayerDataTwoDataTwo data...
	✔ GlobalVictoryVictory data...
	✔ Diplomacyplomacy data...
	✔ OptionsOptions data...
	✔ Maping Map data...
	✔ Unitsg Units data...
	✔ Triggersriggers data...
	✔ Filesg Files data...
[21:11:20] Parsing scenario file finished successfully.

[21:11:20] Setting up managers ...
	✔ MessageManagereManager...
	✔ PlayerManagerrManager...
	✔ MapManagerpManager...
	✔ UnitManagertManager...
	✔ TriggerManagerrManager...
	✔ XsManagersMa

## 触发器重新排序
如果场景中触发器顺序被打乱了可能有必要对其重新排序。

#### 关于触发器 `创建顺序ID` 和 `显示顺序ID`
- **创建顺序ID**: 该触发器在场景中被创建的顺序，即第几次点击"新建"触发按钮，该触发器的创建顺序就是几，创建顺序就是触发器的触发ID（trigger.trigger_id）<font color="blue"><font>

- **显示顺序ID**: 该触发器在“触发”选项卡的UI面板中，自上而下的排列顺序（如上图中`T7`触发显示序ID为2, `T5`触发显示序ID为4）


In [4]:
# 触发器列表的显示顺序
show_orders = trigger_mgr.trigger_display_order
print("显示序：", show_orders)

显示序： [0, 5, 4, 2, 3, 1]


In [5]:
# 触发器列表的创建顺序
create_orders = []

print("显示序\t触发ID\t触发名称")
print("------------------------------")
for idx,tid in enumerate(show_orders):
    trigger = trigger_mgr.triggers[tid]
    create_orders.append(tid);
    print(idx, "\t", tid, '\t', trigger.name)


显示序	触发ID	触发名称
------------------------------
0 	 0 	 ***** 触发器删除和重排 *****
1 	 5 	 XS函数定义
2 	 4 	 T7
3 	 2 	 T3
4 	 3 	 T5
5 	 1 	 T1


## 按照显示顺序重排触发器
**显示顺序即列表 `show_orders` 的索引，根据该搜索引对触发器列表顺序进行重新排列。**

<font color="red">**对乱序的触发进行重新排序，可以解决以下问题：**</font>
- <font color="blue">解决触发器ID混乱导致的触发器跨图移植不成功（触发器跨图移植，请参考第3部分文档） </font>
- <font color="blue">若新建的用于定义XS功能的触发器，创建序位于调用该脚本的触发器之后，会产生错误（常见于新增的XS功能能定义）。通过重排触发器能解决这个依赖问题。</font>
- <font color="blue">重排触发器有利于作者维护地图触发，并提升地图性能。</font>

## 对乱序的触发器ID按照显示序重排

In [6]:
# 触发器重排
trigger_mgr.reorder_triggers(trigger_mgr.trigger_display_order)

In [7]:
# 重排后查看触发显示序和创建序是否一致
del show_orders
show_orders = trigger_mgr.trigger_display_order
print("显示序\t触发ID\t触发名称")
print("------------------------------")
for idx,tid in enumerate(show_orders):
    trigger = trigger_mgr.triggers[tid]
    create_orders.append(tid);
    print(idx, "\t", tid, '\t', trigger.name)


显示序	触发ID	触发名称
------------------------------
0 	 0 	 ***** 触发器删除和重排 *****
1 	 1 	 XS函数定义
2 	 2 	 T7
3 	 3 	 T3
4 	 4 	 T5
5 	 5 	 T1


In [8]:
# 将排序号的场景保存
scenario.write_to_file(des_path)


[21:17:33] Reconstructing sections and structs from managers...
	✔ MessageManagerssageManager...
	✔ PlayerManagerlayerManager...
	✔ MapManagerg MapManager...
	✔ UnitManager UnitManager...
	✔ TriggerManageriggerManager...
	✔ XsManagerng XsManager...
	✔ OptionManagerptionManager...
[21:17:33] Reconstruction finished successfully.

[21:17:33] File writing from structure started...
	✔ FileHeaderg FileHeader...
	✔ DataHeaderg DataHeader...
	✔ Messagesing Messages...
	✔ Cinematicsg Cinematics...
	✔ BackgroundImagekgroundImage...
	✔ PlayerDataTwolayerDataTwo...
	✔ GlobalVictorylobalVictory...
	✔ Diplomacyng Diplomacy...
	✔ Optionsting Options...
	✔ Maptructing Map...
	✔ Unitsucting Units...
	✔ Triggersing Triggers...
	✔ Filesucting Files...
[21:17:33] File writing finished successfully.
[21:17:33] File successfully written to: 'C:/Users/babycat/Games/Age of Empires 2 DE/76561198386517457/resources/_common/scenario/TEST-触发器排序.aoe2scenario'
[21:17:33] Execution time from scenario read: 373.4s


## <font color="green">触发器重排的函数封装</font>

In [ ]:
def reorder_scx_triggers(scx_path: str, output_path: str, show_order: bool=True):
    '''
    Args: 
        scx_path: The absolute/relative path of your scenario which need to reorder triggers.
        output_path: The absolute/relative save path of reorded scenario.
        show_order: If set True, will show trigger's `display_orders` and `create_orders`
    '''
    scenario = AoE2DEScenario.from_file(scx_path)
    trigger_mgr = scenario.trigger_manager
    trigger_amt = trigger_mgr.triggers.__len__()
    if trigger_amt == 0:
        print("Scenario cantains 0 triggers, need not reorder triggers.")
        return None
    elif trigger_amt == 1:
        print("Scenario cantains 1 triggers, need not reorder triggers.")
        return None
    else:
        # Get display_orders and create_orders
        display_orders = []
        create_orders = []
        for idx,tid in enumerate(trigger_mgr.trigger_display_order):
            display_orders.append(idx)
            create_orders.append(tid)
        if show_order:
            print("Show Order\tTrigger ID\tTrigger Name")
            print("------------------------------")
            for idx,tid in zip(display_orders, create_orders):
                trigger = trigger_mgr.triggers[tid]
                print(idx, "\t", tid, '\t', trigger.name)
        
        if display_orders == create_orders:
            # don't reorder trigger
            print("Trigger displayer_orders are equal to create_orders, need not reorder triggers.")
            return None
        else:
            print("Start reorder triggers ......")
            trigger_mgr.reorder_triggers(trigger_mgr.trigger_display_order)
            # save scenario
            scenario.write_to_file(output_path)
            print("Reorder triggers finished !")
